In [13]:
import pandas as pd
import plotly.express as px

In [14]:
spacex_df=pd.read_csv('/Users/sjroh/IBM_DataScience_Certificate/Capstone_Project/spacex_launch_dash.csv')
spacex_df.head(2)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0


In [15]:
filtered_df = spacex_df[spacex_df['Launch Site'] == 'CCAFS LC-40'].value_counts('class').to_frame().reset_index()
filtered_df

,class,0
0,0,19
1,1,7


In [16]:
px.pie(filtered_df, values = 0, names = 'class')

In [17]:
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class',
        names = 'Launch Site',
        title = 'Total Success Launches by Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == str(entered_site)].value_counts('class').to_frame().reset_index()
        fig1 = px.pie(filtered_df,
        values= 0,
        names = 'class',
        title = 'Total Success Launches for site ' + str(entered_site))
        return fig1    

In [18]:
get_pie_chart('CCAFS LC-40')

In [19]:
def get_filtered_scatter_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.scatter(spacex_df, x='Payload Mass (kg)', y='class',
        color='Booster Version Category',
        title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        filtered_df3 = spacex_df[spacex_df['Launch Site'] == str(entered_site)]
        fig1 = px.scatter(filtered_df3, x='Payload Mass (kg)', y='class',
        color='Booster Version Category',
        title='Correlation between Payload and Success for ' + str(entered_site) + ' Site')
        return fig1

In [20]:
get_filtered_scatter_chart('CCAFS LC-40')

In [21]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',options=[{'label': 'All Sites', 'value': 'ALL'},
                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'}, 
                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                value= 'ALL',
                                placeholder= 'All Sites',
                                searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                min=0, max=10000, step=1000,
                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                html.Br()
                                ])
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class',
        names = 'Launch Site',
        title = 'Total Success Launches by Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == str(entered_site)].value_counts('class').to_frame().reset_index()
        fig1 = px.pie(filtered_df, 
        values= 0,
        names = 'class',
        title = 'Total Success Launches for site ' + str(entered_site))
        return fig1    

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')])


def get_filtered_scatter_chart(entered_site, payload):
    filtered2_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]
    if entered_site == 'ALL':
        fig = px.scatter(filtered2_df, x='Payload Mass (kg)', y='class',
        color='Booster Version Category',
        title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        fig1 = px.scatter(filtered2_df[filtered2_df['Launch Site'] == str(entered_site)], 
        x='Payload Mass (kg)', y='class',
        color='Booster Version Category',
        title='Correlation between Payload and Success for ' + str(entered_site) + ' Site')
        return fig1

# Run the app
if __name__ == '__main__':
    app.run_server()


